<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/%EC%95%99%EC%83%81%EB%B8%94(%EB%B3%B4%ED%8C%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Labels_Safe_MSR+Juliet_no_random.csv의 사본')

In [5]:
df

,Unnamed: 0,CWE ID,lang,vul,code,labels
0,115.0,safe,c,0,"void func(fz_context *ctx, fz_color_converter ...",0
1,116.0,safe,c,0,"void func(fz_context *ctx, fz_color_converter ...",0
2,117.0,safe,c,0,"void func(fz_context *ctx, const fz_colorspace...",0
3,118.0,safe,c,0,static inline void func(unsigned char *restric...,0
4,119.0,safe,c,0,"void func(const fz_colorspace *cs, const float...",0
...,...,...,...,...,...,...
16396,NaN,CWE-416,cpp,1,void func() {\n\t wchar_t * data; /* Ini...,8
16397,NaN,CWE-416,cpp,1,void func() {\n\t wchar_t * data; /* Ini...,8
16398,NaN,CWE-121,c,1,void func() {\n\t wchar_t * data; other_...,4
16399,NaN,CWE-121,c,1,static void func(wchar_t * data) {\n\t {\n...,4


In [6]:
df.columns

Index(['Unnamed: 0', 'CWE ID', 'lang', 'vul', 'code', 'labels'], dtype='object')

In [7]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [9]:
model_names = [
    "microsoft/graphcodebert-base",
    "microsoft/codebert-base-mlm",
    "microsoft/unixcoder-base"
]

models = [AutoModelForSequenceClassification.from_pretrained(name, num_labels=11) for name in model_names]
tokenizers = [AutoTokenizer.from_pretrained(name) for name in model_names]

batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/unixcoder-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/938k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/444k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [10]:

df = df.drop(columns=["vul", "Unnamed: 0"])

In [16]:
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from datasets import load_dataset

df['code'] = df['code'].apply(lambda x: re.sub(r'/\*.*?\*/', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'//.*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include <.*?>\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include \".*?\"\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#define .*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'[\t ]+', ' ', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'\n\s*\n', '\n', x))
df.to_csv("preprocessed_data.csv", index=False)

dataset = load_dataset("csv", data_files="preprocessed_data.csv")['train']



Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
"""# 데이터 전처리
preprocessed_df = preprocess_code(df)

# "code","labels" 컬럼을 제외한 열 선택
columns_to_encode = [col for col in preprocessed_df.columns if col not in ["code", "labels"]]

# 원핫인코딩 수행
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(preprocessed_df[columns_to_encode])

# 인코딩된 데이터를 데이터프레임으로 변환
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(columns_to_encode))

# "code" 컬럼과 인코딩된 데이터 결합
result_df = pd.concat([preprocessed_df[["code", "labels"]], encoded_df], axis=1)

# 데이터프레임을 CSV 파일로 저장
result_df.to_csv("preprocessed_data.csv", index=False)"""


In [19]:
# 데이터셋 로드 및 전처리
dataset = load_dataset("csv", data_files="preprocessed_data.csv")['train']

In [21]:
from sklearn.model_selection import KFold
import torch
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

# EarlyStoppingCallback 설정
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=5,  # 성능 개선이 3 에폭 동안 관찰되지 않으면 중단
    early_stopping_threshold=0.1  # 개선된 것으로 간주하기 위한 최소 변경량
)

# K-Fold 교차 검증 설정
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [ ]:
"""from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics"""


In [36]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    weight_decay=0.01
)


In [39]:
# 앙상블 및 성능 평가
all_f1_scores = []
fold_results = []

In [37]:
from sklearn.metrics import f1_score
def tokenize_and_convert_to_tensor(examples, tokenizer):
    return tokenizer(examples['code'], max_length=512, padding="max_length", truncation=True, return_tensors="pt")

# 데이터셋 분할 및 토큰화
for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    train_subset = dataset.select(train_idx)
    val_subset = dataset.select(val_idx)
    tokenized_train_datasets = [train_subset.map(lambda x: tokenize_and_convert_to_tensor(x, tokenizer), batched=True) for tokenizer in tokenizers]
    tokenized_val_datasets = [val_subset.map(lambda x: tokenize_and_convert_to_tensor(x, tokenizer), batched=True) for tokenizer in tokenizers]

    for model, train_data, val_data in zip(models, tokenized_train_datasets, tokenized_val_datasets):
        trainer = Trainer(model=model, args=training_args, train_dataset=train_data, eval_dataset=val_data)
        trainer.train()

Epoch,Training Loss,Validation Loss
1,0.046300,0.942481


Epoch,Training Loss,Validation Loss
1,0.327800,0.570706


Epoch,Training Loss,Validation Loss
1,0.364600,0.526743


Map:   0%|          | 0/13121 [00:00<?, ? examples/s]

Map:   0%|          | 0/13121 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3280 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.198100,0.131753


Epoch,Training Loss,Validation Loss
1,0.333000,0.211151


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.278700,0.172323


In [48]:
models_trained = []  # 현재 폴드에서 훈련된 모델들을 저장
models_trained.append(model)
fold_results.append((models_trained, tokenized_val_datasets, val_subset['labels']))

# 결과 저장
torch.save(fold_results, 'fold_training_results.pth')

In [61]:
fold_results = torch.load('/content/drive/MyDrive/fold_training_results.pth')


OutOfMemoryError: CUDA out of memory. Tried to allocate 152.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 16.38 MiB is free. Process 49398 has 15.75 GiB memory in use. Of the allocated memory 15.30 GiB is allocated by PyTorch, and 82.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [59]:
import torch
from sklearn.metrics import f1_score

all_f1_scores = []

for models_trained, tokenized_val_datasets, labels in fold_results:
    fold_predictions = []

    for model, val_data in zip(models_trained, tokenized_val_datasets):
        # 모든 데이터를 한 번에 모델의 입력 형식에 맞게 준비
        input_ids = torch.tensor([x['input_ids'] for x in val_data]).to(model.device)
        attention_mask = torch.tensor([x['attention_mask'] for x in val_data]).to(model.device)

        # 모델 실행
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        fold_predictions.append(probabilities)

    # 앙상블 예측 및 평가 계산
    fold_ensemble_pred = torch.mean(torch.stack(fold_predictions), dim=0)
    final_pred = torch.argmax(fold_ensemble_pred, dim=-1).numpy()

    # F1 스코어 계산
    f1_macro = f1_score(labels.numpy(), final_pred, average='macro')
    all_f1_scores.append(f1_macro)
    print(f"F1 Macro Score: {f1_macro}")

average_f1_macro = np.mean(all_f1_scores)
print(f"Average F1 Macro Score: {average_f1_macro}")


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.80 GiB. GPU 0 has a total capacity of 15.77 GiB of which 16.38 MiB is free. Process 49398 has 15.75 GiB memory in use. Of the allocated memory 15.30 GiB is allocated by PyTorch, and 82.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)